In [88]:
from bs4 import BeautifulSoup
import os
import codecs
import xml.etree.ElementTree as ET
import pandas as pd

In [120]:
soup = BeautifulSoup(codecs.open(f"{os.getcwd()}/invoices/NOR170329J92_3122.xml","r", encoding='utf-8',errors='ignore'), 'xml')
ssoup.find("cfdi:Comprobante").get('Certificado')

'MIIF7DCCA9SgAwIBAgIUMDAwMDEwMDAwMDA1MDc0NzEzMzgwDQYJKoZIhvcNAQELBQAwggGEMSAwHgYDVQQDDBdBVVRPUklEQUQgQ0VSVElGSUNBRE9SQTEuMCwGA1UECgwlU0VSVklDSU8gREUgQURNSU5JU1RSQUNJT04gVFJJQlVUQVJJQTEaMBgGA1UECwwRU0FULUlFUyBBdXRob3JpdHkxKjAoBgkqhkiG9w0BCQEWG2NvbnRhY3RvLnRlY25pY29Ac2F0LmdvYi5teDEmMCQGA1UECQwdQVYuIEhJREFMR08gNzcsIENPTC4gR1VFUlJFUk8xDjAMBgNVBBEMBTA2MzAwMQswCQYDVQQGEwJNWDEZMBcGA1UECAwQQ0lVREFEIERFIE1FWElDTzETMBEGA1UEBwwKQ1VBVUhURU1PQzEVMBMGA1UELRMMU0FUOTcwNzAxTk4zMVwwWgYJKoZIhvcNAQkCE01yZXNwb25zYWJsZTogQURNSU5JU1RSQUNJT04gQ0VOVFJBTCBERSBTRVJWSUNJT1MgVFJJQlVUQVJJT1MgQUwgQ09OVFJJQlVZRU5URTAeFw0yMTA1MjExNDM3MzBaFw0yNTA1MjExNDM3MzBaMIG6MR4wHAYDVQQDExVOT1RBUklPUyBPUkdBTklTVEEgU0MxHjAcBgNVBCkTFU5PVEFSSU9TIE9SR0FOSVNUQSBTQzEeMBwGA1UEChMVTk9UQVJJT1MgT1JHQU5JU1RBIFNDMSUwIwYDVQQtExxOT1IxNzAzMjlKOTIgLyBPQVBBNzMwMTEwN1UwMR4wHAYDVQQFExUgLyBPQVBBNzMwMTEwSEpDUlJMMDAxETAPBgNVBAsTCHVuaWRhZCAxMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvBFbEMPXYmNJekCoCaQAoBcZO0EW6BUVkzftStTEvp9UgITLE2R9hF3wCJIG7unscGm6RDC

In [115]:
class Factura():

    # Constructor of an invoice object
    def __init__(self, path):
        self.path = path
        self.soup = BeautifulSoup(codecs.open(self.path,"r", encoding='utf-8',errors='ignore'), 'xml')

        self.xsi_schema_location = soup.find("cfdi:Comprobante").get("xsi:schemaLocation")
        self.xmlns_tfd = soup.find("cfdi:Comprobante").get("xmlns:tfd")
        self.xmlns_notariospublicos = soup.find("cfdi:Comprobante").get("xmlns:notariospublicos")
        self.xmlns_xsi = soup.find("cfdi:Comprobante").get("xmlns:xsi")        
        self.version = soup.find("cfdi:Comprobante").get("Version")
        self.folio = soup.find("cfdi:Comprobante").get("Folio")
        self.fecha = soup.find("cfdi:Comprobante").get("Fecha")
        self.sello = soup.find("cfdi:Comprobante").get("Sello")
        self.formaPago = soup.find("cfdi:Comprobante").get("FormaPago")
        self.NoCertificado = soup.find("cfdi:Comprobante").get("NoCertificado")
        self.Certificado = soup.find("cfdi:Comprobante").get("Certificado")
        self.subTotal = soup.find("cfdi:Comprobante").get("SubTotal")
        self.moneda = soup.find("cfdi:Comprobante").get("Moneda")
        self.tipoCambio = soup.find("cfdi:Comprobante").get("TipoCambio")
        self.total = soup.find("cfdi:Comprobante").get("Total")
        self.tipoDeComprobante = soup.find("cfdi:Comprobante").get("TipoDeComprobante")
        self.exportacion = soup.find("cfdi:Comprobante").get("Exportacion")
        self.metodoPago = soup.find("cfdi:Comprobante").get("MetodoPago")
        self.lugarExpedicion = soup.find("cfdi:Comprobante").get("LugarExpedicion")
        self.xmlns_cfdi = soup.find("cfdi:Comprobante").get("xmlns:cfdi")


        self.emisor = Emisor(self.soup.find('cfdi:Emisor'))
        self.receptor = Receptor(self.soup.find('cfdi:Receptor'))
        self.productos = pd.DataFrame(columns=
                                      ["xsi_schema_location","xmlns_tfd","xmlns_notariospublicos","xmlns_xsi","version","folio","fecha","sello"
                                       ,"formaPago","NoCertificado" ,"Certificado","subTotal","moneda","tipoCambio","total" ,"tipoDeComprobante"
                                       ,"exportacion","metodoPago","lugarExpedicion","xmlns_cfdi"])
        
        for concepto in self.soup.find('cfdi:Conceptos') .find_all('cfdi:Concepto'):
            self.productos.loc[len(self.productos)] = [concepto.get('ClaveProdServ')
                                                        ,concepto.get('Cantidad')
                                                        ,concepto.get('ClaveUnidad')
                                                        ,concepto.get('Unidad')
                                                        ,concepto.get('Unidad de servicio')
                                                        ,concepto.get('Descripcion')
                                                        ,concepto.get('ValorUnitario')
                                                        ,concepto.get('Importe')
                                                        ,concepto.get('ObjetoImp')
                                                        ]
            

class Emisor():
    
        # Constructor of an Emisor object
        def __init__(self, element):
            self.nombre = element.get('Nombre')
            self.rfc = element.get('Rfc')
            self.regimen = element.get('RegimenFiscal')

class Receptor():
    
        # Constructor of an Receptor object
        def __init__(self, element):
            self.nombre = element.get('Nombre')
            self.rfc = element.get('Rfc')
            self.regimen = element.get('RegimenFiscalReceptor')
            self.uso = element.get('UsoCFDI')
            self.domicilio = element.get('DomicilioFiscalReceptor')




In [116]:
Factura(f"{os.getcwd()}/invoices/NOR170329J92_3122.xml").comprobante


<cfdi:Comprobante Certificado="MIIF7DCCA9SgAwIBAgIUMDAwMDEwMDAwMDA1MDc0NzEzMzgwDQYJKoZIhvcNAQELBQAwggGEMSAwHgYDVQQDDBdBVVRPUklEQUQgQ0VSVElGSUNBRE9SQTEuMCwGA1UECgwlU0VSVklDSU8gREUgQURNSU5JU1RSQUNJT04gVFJJQlVUQVJJQTEaMBgGA1UECwwRU0FULUlFUyBBdXRob3JpdHkxKjAoBgkqhkiG9w0BCQEWG2NvbnRhY3RvLnRlY25pY29Ac2F0LmdvYi5teDEmMCQGA1UECQwdQVYuIEhJREFMR08gNzcsIENPTC4gR1VFUlJFUk8xDjAMBgNVBBEMBTA2MzAwMQswCQYDVQQGEwJNWDEZMBcGA1UECAwQQ0lVREFEIERFIE1FWElDTzETMBEGA1UEBwwKQ1VBVUhURU1PQzEVMBMGA1UELRMMU0FUOTcwNzAxTk4zMVwwWgYJKoZIhvcNAQkCE01yZXNwb25zYWJsZTogQURNSU5JU1RSQUNJT04gQ0VOVFJBTCBERSBTRVJWSUNJT1MgVFJJQlVUQVJJT1MgQUwgQ09OVFJJQlVZRU5URTAeFw0yMTA1MjExNDM3MzBaFw0yNTA1MjExNDM3MzBaMIG6MR4wHAYDVQQDExVOT1RBUklPUyBPUkdBTklTVEEgU0MxHjAcBgNVBCkTFU5PVEFSSU9TIE9SR0FOSVNUQSBTQzEeMBwGA1UEChMVTk9UQVJJT1MgT1JHQU5JU1RBIFNDMSUwIwYDVQQtExxOT1IxNzAzMjlKOTIgLyBPQVBBNzMwMTEwN1UwMR4wHAYDVQQFExUgLyBPQVBBNzMwMTEwSEpDUlJMMDAxETAPBgNVBAsTCHVuaWRhZCAxMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvBFbEMPXYmNJekCoCaQAoBcZO0EW6BUVkzftStTEv

In [73]:
import codecs
with codecs.open(f"{os.getcwd()}/invoices/000000586751825.xml", 'r', encoding='utf-8',errors='ignore') as f:
    data = f.read()

soup = BeautifulSoup(data, "xml")
soup.prettify()

'<?xml version="1.0" encoding="utf-8"?>\n<cfdi:Comprobante Certificado="MIIGRDCCBCygAwIBAgIUMDAwMDEwMDAwMDA1MTI5NzkyNjAwDQYJKoZIhvcNAQELBQAwggGEMSAwHgYDVQQDDBdBVVRPUklEQUQgQ0VSVElGSUNBRE9SQTEuMCwGA1UECgwlU0VSVklDSU8gREUgQURNSU5JU1RSQUNJT04gVFJJQlVUQVJJQTEaMBgGA1UECwwRU0FULUlFUyBBdXRob3JpdHkxKjAoBgkqhkiG9w0BCQEWG2NvbnRhY3RvLnRlY25pY29Ac2F0LmdvYi5teDEmMCQGA1UECQwdQVYuIEhJREFMR08gNzcsIENPTC4gR1VFUlJFUk8xDjAMBgNVBBEMBTA2MzAwMQswCQYDVQQGEwJNWDEZMBcGA1UECAwQQ0lVREFEIERFIE1FWElDTzETMBEGA1UEBwwKQ1VBVUhURU1PQzEVMBMGA1UELRMMU0FUOTcwNzAxTk4zMVwwWgYJKoZIhvcNAQkCE01yZXNwb25zYWJsZTogQURNSU5JU1RSQUNJT04gQ0VOVFJBTCBERSBTRVJWSUNJT1MgVFJJQlVUQVJJT1MgQUwgQ09OVFJJQlVZRU5URTAeFw0yMjA1MTMxNzAzNTdaFw0yNjA1MTMxNzAzNTdaMIIBETE0MDIGA1UEAxQrQVQmVCBDT01VTklDQUNJT05FUyBESUdJVEFMRVMgUyBERSBSTCBERSBDVjE0MDIGA1UEKRQrQVQmVCBDT01VTklDQUNJT05FUyBESUdJVEFMRVMgUyBERSBSTCBERSBDVjE0MDIGA1UEChQrQVQmVCBDT01VTklDQUNJT05FUyBESUdJVEFMRVMgUyBERSBSTCBERSBDVjElMCMGA1UELRMcQ05NOTgwMTE0UEkyIC8gTFVISTcyMDEzMUtaNjEeMBwGA1UEBRMVIC8gTFVI